### Import essential libraries and create a Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

25/04/08 17:26:03 WARN Utils: Your hostname, bigdata resolves to a loopback address: 127.0.1.1; using 172.22.193.31 instead (on interface eth0)
25/04/08 17:26:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/08 17:26:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv('asr.csv',header= True)

In [3]:
df.show(5)

+-----+-------------------+--------+--------------------+----------+--------------+------------------+-------+---------------+-------------+----+----------+--------------+---+--------+------+-----------+-----------+----------------+------------+--------------------+-----+------------+-----------+
|index|           Order ID|    Date|              Status|Fulfilment|Sales Channel |ship-service-level|  Style|            SKU|     Category|Size|      ASIN|Courier Status|Qty|currency|Amount|  ship-city| ship-state|ship-postal-code|ship-country|       promotion-ids|  B2B|fulfilled-by|Unnamed: 22|
+-----+-------------------+--------+--------------------+----------+--------------+------------------+-------+---------------+-------------+----+----------+--------------+---+--------+------+-----------+-----------+----------------+------------+--------------------+-----+------------+-----------+
|    0|405-8078784-5731545|04-30-22|           Cancelled|  Merchant|     Amazon.in|          Standard| SET

In [4]:
df.printSchema()

root
 |-- index: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Sales Channel : string (nullable = true)
 |-- ship-service-level: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier Status: string (nullable = true)
 |-- Qty: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- ship-city: string (nullable = true)
 |-- ship-state: string (nullable = true)
 |-- ship-postal-code: string (nullable = true)
 |-- ship-country: string (nullable = true)
 |-- promotion-ids: string (nullable = true)
 |-- B2B: string (nullable = true)
 |-- fulfilled-by: string (nullable = true)
 |-- Unnamed: 22: string (nullable = true)



In [5]:
df = df.filter(F.lower(F.col("Status")).contains("shipped"))

In [ ]:
df = df.withColumn("Date", F.to_date("Date","MM-dd-yy"))

In [7]:
df = df.withColumn("report_date", F.next_day(F.col("Date"), "Mon"))

In [9]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

result_df = df.groupBy("report_date", "SKU") \
    .agg(F.sum("Qty").alias("total_quantity")) \
    .withColumnRenamed("SKU", "sku") \
    .orderBy("report_date", "SKU")
    

result_df = result_df.withColumn("total_quantity", col("total_quantity").cast(IntegerType()))

In [10]:
result_df.show()

+-----------+---------------+--------------+
|report_date|            sku|total_quantity|
+-----------+---------------+--------------+
| 2022-04-04| AN209-BIEGE-XL|             1|
| 2022-04-04|BL003-50BLACK-B|             1|
| 2022-04-04|  BL009-61BLACK|             3|
| 2022-04-04|   BL015-63PINK|             1|
| 2022-04-04| BL023-74PINK-B|             1|
| 2022-04-04|  BL035-161GOLD|             4|
| 2022-04-04|BL057-65BLACK-A|             3|
| 2022-04-04|        BL099-L|             1|
| 2022-04-04|        BL102-L|             1|
| 2022-04-04|       BL103-XS|             1|
| 2022-04-04|      BL104-XXL|             1|
| 2022-04-04|        BL110-L|             1|
| 2022-04-04|        BL111-L|             2|
| 2022-04-04|        BL111-M|             1|
| 2022-04-04|        BL113-S|             1|
| 2022-04-04|       BL113-XS|             1|
| 2022-04-04|    BTM029-NP-L|             1|
| 2022-04-04|    BTM029-NP-S|             1|
| 2022-04-04|   BTM032-NP-XS|             1|
| 2022-04-

### Verify

In [ ]:
main_df = spark.read.csv('../../src/Task_2.2/output.csv',header= True)

main_df.show()

+-----------+---------------+--------------+
|report_date|            sku|total_quantity|
+-----------+---------------+--------------+
| 2022-04-04| AN209-BIEGE-XL|             1|
| 2022-04-04|BL003-50BLACK-B|             1|
| 2022-04-04|  BL009-61BLACK|             3|
| 2022-04-04|   BL015-63PINK|             1|
| 2022-04-04| BL023-74PINK-B|             1|
| 2022-04-04|  BL035-161GOLD|             4|
| 2022-04-04|BL057-65BLACK-A|             3|
| 2022-04-04|        BL099-L|             1|
| 2022-04-04|        BL102-L|             1|
| 2022-04-04|       BL103-XS|             1|
| 2022-04-04|      BL104-XXL|             1|
| 2022-04-04|        BL110-L|             1|
| 2022-04-04|        BL111-L|             2|
| 2022-04-04|        BL111-M|             1|
| 2022-04-04|        BL113-S|             1|
| 2022-04-04|       BL113-XS|             1|
| 2022-04-04|    BTM029-NP-L|             1|
| 2022-04-04|    BTM029-NP-S|             1|
| 2022-04-04|   BTM032-NP-XS|             1|
| 2022-04-

In [14]:
# Dòng có trong df1 nhưng không có trong df2
diff1 = main_df.subtract(result_df)

# Dòng có trong df2 nhưng không có trong df1
diff2 = result_df.subtract(main_df)

# Union 2 phần khác nhau lại để thấy toàn bộ dòng khác nhau
differences = diff1.union(diff2)

# In ra kết quả
differences.show()

print(differences.count())


+-----------+---+--------------+
|report_date|sku|total_quantity|
+-----------+---+--------------+
+-----------+---+--------------+



0
